In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from pasna_analysis import DataLoader, Experiment, utils

experiment_name = '20240515'
exp_path = Path.cwd().parent.joinpath('data',experiment_name)
exp = Experiment(DataLoader(exp_path), first_peak_threshold=30)

In [ ]:
'''Plots activity signals for a group of embryos.'''
# Change start and end to select different groups
start = 3
end = 6

fig, ax = plt.subplots(end-start, 2, figsize=(12, 4*(end-start)))
for i, emb in enumerate(exp.embryos[start:end]):
    time = emb.activity[:, 0]
    trace = exp.traces[emb.name]
    ax[i][0].set_title(f'{emb.name} - GCamP')
    ax[i][0].plot(time, trace.active, color='green')
    ax[i][0].axvline(time[trace.trim_idx], color='k', linestyle='--')
    ax[i][1].set_title(f'{emb.name} - tdTomato')
    ax[i][1].plot(time, trace.struct, color='firebrick')
    ax[i][1].axvline(time[trace.trim_idx], color='k', linestyle='--')

    for peak in trace.peak_times:
        ax[i][0].axvline(peak, color='k')

In [ ]:
'''Plots dff for a group of embryos.'''
# Change start and end to select different groups
start = 3
end = 6

fig, ax = plt.subplots(end-start, figsize=(8, 4*(end-start)))
for i, emb in enumerate(exp.embryos[start:end]):
    time = emb.activity[:, 0]
    trace = exp.traces[emb.name]
    time = time[:trace.trim_idx]
    ax[i].set_title(f'{emb.name} - dff')
    ax[i].plot(time, trace.dff)
    for peak in trace.peak_times:
        ax[i].axvline(peak, color='k', alpha=0.3)

In [ ]:
'''Plots AUC, grouped by bins.'''
data = {'auc': [], 'bin': [], 'emb': []}

n_bins = 5
first_bin = 2
bin_width = 0.2
for i, emb in enumerate(exp.embryos):
    trace = exp.traces[emb.name]

    dev_time_at_peaks = emb.get_DT_from_time(trace.peak_times)

    bins = [first_bin + j*bin_width for j in range(n_bins)]
    bin_idxs = utils.split_in_bins(dev_time_at_peaks, bins)

    data['auc'].extend(trace.peak_aucs)
    data['bin'].extend(bin_idxs)
    data['emb'].extend([str(i)]*len(trace.peak_aucs))

# add a last bin point to generate the labels
bins.append(first_bin + bin_width*n_bins)
x_labels = [f"{s}~{e}" for (s, e) in zip(bins[:-1], bins[1:])]
ax = sns.pointplot(data=data, x='bin', y='auc', linestyle='None')
ax.set_xticks(ticks=list(range(n_bins)), labels=x_labels)
ax.set_title(f'Binned AUC - Exp {exp.name}')
ax.set_ylabel('AUC [activity*t]');